Envio de Mensgaens no Whatsapp com Python

                        Considerações

-> 1. O whatsapp não gosta de nenhum tipo de automação.

-> 2. Isso não é o uso da API oficial do whatsapp, o próprio whatsapp
tem uma API oficial. Se o seu objetivo é envio em massa ou criar aqueles
robozinhos que respondem automaticamente no whatsapp, então use a API
oficial.

-> O objetivo do exercicio é 100% educacional

In [ ]:
# Selenium Biblioteca que possui um conjunto de funções especificas para automação de navegadores web
# (ações como clicar, digitar, preencher formulários, etc)

# Import do módulo WebDriver que possui os drives que possibilitam o acesso do python a navegadores web através de funções e classes.
from selenium import webdriver

# Import da classe By que tem como objetivo selecionar a forma como
# o selenium irá encontrar os elementos que queremos acessar na página.
# Basicamente devemos passa-lo como argumento dentro do find element com o objetivo de escolher o atributo (pertencente a classe by) que vamos utilizar na seleção do elemento.
from selenium.webdriver.common.by import By 

# Biblioteca que tem como objetivo manipular o tempo de execução do sistema.
import time

Ira Entrar no Whatsapp

In [ ]:
# 
#Instância da classe Chrome que irá criar o nosso navegador
navegador = webdriver.Chrome()

# Get: Método da classe Chrome que tem como objetivo acessar sites nos navegadores usando o link da página como argumento.
navegador.get("https://web.whatsapp.com")

# Essa etapa é importantissima, pois, para que as automações funcionem, devemos 
# garantir que a página acessada esteja 100% carregada antes do python executar o
# bloco de código. Caso contrário, o código ira dar erro já que o python não encontrou
# os elementos que ele deveria acessar. Para resolver isso, criamos um loop while que
# a cada 1 segundo irá verificar se a barra lateral ja apareceu no sistema. Dessa forma
# o python sabera se a página ja foi ou não carregada completamente, pois, caso o site ainda
# não esteja pronto, o python voltara para o while para realizar a verificação da existência 
# da barra lateral. Resumindo de 1 em 1 segundo ele vai verificar se a página está pronta para 
# seguir com a automação.

# find_elements: Método da classe Chrome que tem como objetivo procurar e selecionar elementos de uma página, ele recebe como argumento:
# O By.ID: Atributo da classe By que seleciona IDS de elementos HTML
# side: ID da barra lateral do whatsapp
# While: Ira verificar se a lista de elementos com o id 'side' é menor que 1.
# Se essa condição for verdadeira, ele irá aguardar 1 segundo e realizar a
# verificação da lista novamente.
while len(navegador.find_elements(By.ID, 'side')) < 1:
        
        time.sleep(1)

Ira acessar o arquivo que contém os telefones, as mensagens, o nome do contato
e o arquivo que deve ser enviado 

In [ ]:
# Import da biblioteca pandas que tem como objetivo manipular e análisar
# base de dados. Iremos utilizar essa biblioteca para acessar a planilha
# que contém os dados das pessoas que receberam a mensagem.   
import pandas as pd

# read_excel: Função da biblioteca pandas que tem como objetivo acessar os
# valores de uma planilha. Ela recebe como argumento o caminho da planilha
# que será acessada. Observação: Como a planilha está na mesma pasta do
# nosso exercicio, precisamos passar apenas o nome e a extensão do arquivo.
dados = pd.read_excel('Envios.xlsx')

# Função do pandas que tem como objetivo apagar valores (linhas) nulos de uma base de dados.
dados = dados.dropna()

# Como a planilha possui dados sensiveis (o telefone das pessoas) 
# vamos filtrar as colunas que devem ser mostrada com o objetivo de
# ocultar as informações da coluna de telefone
# Observação: A unica função dessa variável será mostrar os valores
# da planilha de envios.
exibicao_tabela = dados[['nome', 'mensagem', 'arquivo']]

Exibindo os valores da planilha

In [66]:
print(exibicao_tabela)

       nome                   mensagem                   arquivo
0  Pessoa 1  E aí, fulano, tudo certo?                         N
2  Pessoa 2     tudo certo. Olha isso:  pythonimpressionador.png


Envio de Mensagens

In [ ]:
# Import da bilioteca urllib que tem como objetivo manipular
# urls. Vamos utilizar ela para adaptar o texto da mensagem
# no link da conversa no whatsapp.
import urllib

# Biblioteca que tem como objetivo acessar arquivos no nosso dispositivo.
import os

# For que irá percorrer todas as linhas da planilha com o objetivo
# de coletar os dados presentes em cada coluna. Observação: O
# index é o atributo que contém os indices de cada linha da planilha.
for linha in dados.index:
    
    # Ira conter os nomes da planilha
    nome = dados.loc[linha, "nome"]
    
    # Ira conter as mensagens da planilha
    mensagem = dados.loc[linha, "mensagem"]
    
    # Ira conter os arquivos da planilha
    arquivo = dados.loc[linha, "arquivo"]
    
    # Irá conter os telefones da planilha
    telefone = dados.loc[linha, "telefone"]
    
    # Função nativa do python que irá substituir a palavra "fulano"
    # pelo nome presente na planilha
    texto = mensagem.replace("fulano", nome)
    
    # ira codificar o texto com o objetivo de tratar caracteres especiais
    # que quebram os links. 
    texto = urllib.parse.quote(texto)

    # O link de conversa contém 2 parametros que definem quem vai receber
    # a mensagem:
    # phone: Numero da pessoa que irá receber a mensagem (coluna de telefone)
    # text: Conteúdo da mensagem. (texto da mensagem convertido)
    link = f"https://web.whatsapp.com/send?phone={telefone}&text={texto}"
    
    # Irá imprimir o texto convertido pela urllib
    print(texto)
    
    # Ira pegar o link da conversa que enviará as mensagens para os contatos da planilha
    navegador.get(link)
    
    # Como vimos no começo, irá verificar se a barra lateral representada pelo
    # id side ja apareceu na tela.
    while len(navegador.find_elements(By.ID, 'side')) < 1:
   
        # Intervalo de verificação do loop
        time.sleep(1)
    
    # Intervalo adicional que garante que o tempo de envio mensagem seja realizado
    time.sleep(2)
    
    navegador.find_element(By.XPATH,'//*[@id="main"]/footer/div[1]/div/span/div/div/div/div[4]/div/span/button/div/div/div[1]/span' ).click()
    
    # Intervalo para o inicio de envio de arquivos
    time.sleep(2)
    
    # Ira verificar se a coluna de arquivo possui um valor diferente de "N", ou seja,
    # se a coluna possui o caminho de um arquivo para ser enviado
    if arquivo != "N":
        
        # Ira acessar o caminho absoluto do arquivo que iremos enviar na mensagem usando
        # o a função abspath do sub-módulo path do módulo os. A função recebe como argumento
        # o arquivo da planilha de envios do whatsapp
        caminho_completo = os.path.abspath(f"{arquivo}")
        
        # Ira acessar e clicar no icone de opções de arquivos do whatsapp usando o xpath do elemento
        navegador.find_element(By.XPATH, '//*[@id="main"]/footer/div[1]/div/span/div/div/div/div[1]/div/span/button/div/div/div[1]/span').click()
        
        # Ira acessar o input do tipo "file" e passara para ele como valor o caminho completo do arquivo
        # que queremos enviar ( o metodo send_keys tem como objetivo escrever textos nos elementos selecionados,
        # ou seja, ele irá escrever no input do whatsapp o caminho do arquivo).
        navegador.find_element(By.XPATH, '//*[@id="main"]/footer/input').send_keys(caminho_completo)
        
        # Ira dar um intervalo de 3 segundos para garantir que o arquivo seja completamente carregado
        time.sleep(3)
        
        # Ira clicar no botão de envio do whatsapp
        navegador.find_element(By.XPATH, '//*[@id="app"]/div/div/div[3]/div/div[3]/div[2]/div/span/div/div/div/div[2]/div/div[2]/div[2]/span/div/div/span').click()
        
    # Ira dar um intervalo de 5 segundos antes de encerrar a automação
    time.sleep(5)
    

E%20a%C3%AD%2C%20Pessoa%201%2C%20tudo%20certo%3F
tudo%20certo.%20Olha%20isso%3A
